In [1]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import requests
from bs4 import BeautifulSoup
import time 
from tqdm import tqdm
import random
import pandas as pd

In [2]:
def get_sitemap_urls(sitemap_url):
    Pro=[]
    for i in sitemap_url:
        response = requests.get(i)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            urls = [loc.text for loc in soup.find_all('loc')]
            Pro.extend(urls) 
        else:
            print(f"Failed to fetch sitemap. Status Code: {response.status_code}")
    return Pro     

In [3]:
def unique(Orignal_list):
    unique_list = [item for index, item in enumerate(Orignal_list) if item not in Orignal_list[:index]]
    return unique_list

In [4]:
def scrollWindow():
    time.sleep(2)
    initial_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        driver.implicitly_wait(2) 
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == initial_height:
            break
        initial_height = new_height
       

In [158]:
def get_product_info(URL):
    driver.get(URL)
    scrollWindow()
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    Product = soup.find('h1').text
    ID = soup.find('span', class_="master-product-id").text
    df = []
    for i in range(len(soup.find_all('span', class_="font-weight-bold"))):
        Varient_element = soup.find('ul', class_="sizecode-swatches sizecode-attribute d-flex pt-sm-1 flex-wrap").find_all('li')[i].find_all('span')[0]
        Varient = Varient_element.text
        Stock_status_element = soup.find('ul', class_="sizecode-swatches sizecode-attribute d-flex pt-sm-1 flex-wrap").find_all('li')[i].find_all('span')[1]
        StockStatus = Stock_status_element.text
        try:
            RegularPrice = soup.find_all('span', class_="strike-through list")[i].find('span').get('content')
            SalePrice = soup.find_all('span', class_="sales red").find('span')[i].get('content')
            row = {
                'ProductURL': URL,
                'ProductName':Product,
                'ProductID':ID,
                'Varient':Varient,
                'StockStatus':StockStatus,
                'RegularPrice': '$'+ RegularPrice,
                'SalePrice': '$'+ SalePrice
            }
            df.append(row)
        except Exception as e:
            Price = soup.find_all('span', class_='value')[i].get('content')
            row = {
                'ProductURL': URL,
                'ProductName':Product,
                'ProductID':ID,
                'Varient':Varient,
                'StockStatus':StockStatus,
                'RegularPrice': None,
                'SalePrice': '$'+ Price
            }
            df.append(row)
    return df

In [9]:
allowed_user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Safari/14.1.2",
    "Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)"   
]

In [193]:
sitemap_url = ['https://www.johnnyseeds.com/sitemap_0-product.xml']
sitemap_urls = get_sitemap_urls(sitemap_url)
Pro_URL= []
for i in sitemap_urls:
    if 'https://www.johnnyseeds.com/' in i and 'https://www.johnnyseeds.com/featured/sets-and-collections' not in i :
        Pro_URL.append(i)
df = pd.DataFrame(Pro_URL)
df.to_csv('Links.csv',index=False)

In [194]:
Product_details=[]

In [155]:
Pro_URL.index('https://www.johnnyseeds.com/tools-supplies/clothing/hoodies/mens-zip-hoodie-pacific-l-johnnys-logo-apparel-6671.html')
Pro_URL[3074]

'https://www.johnnyseeds.com/tools-supplies/seed-starting-supplies/soil-blocking/standard-dibbles-soil-blocker-accessory-9527.500.html'

In [196]:
user_agents = random.choice(allowed_user_agents)
headers = {'User-Agent': user_agents}
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(f"user-agent={headers['User-Agent']}")
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
for i in tqdm(Pro_URL[3075:], desc="GettingData", unit="URL"):
    df = get_product_info(i)
    Product_details.extend(df)
    print(df)
daf = pd.DataFrame(Product_details)
daf.to_csv('JohnySeed.csv',index=False)
driver.quit()